In [1]:
!pip show sentence_transformers

Name: sentence-transformers
Version: 3.1.1
Summary: State-of-the-Art Text Embeddings
Home-page: 
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: c:\users\jeanm\appdata\roaming\python\python39\site-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers
Required-by: keybert


In [2]:
!pip show pandas

Name: pandas
Version: 2.0.2
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: 
Author: 
Author-email: The Pandas Development Team <pandas-dev@python.org>
License: BSD 3-Clause License
        
        Copyright (c) 2008-2011, AQR Capital Management, LLC, Lambda Foundry, Inc. and PyData Development Team
        All rights reserved.
        
        Copyright (c) 2011-2023, Open source contributors.
        
        Redistribution and use in source and binary forms, with or without
        modification, are permitted provided that the following conditions are met:
        
        * Redistributions of source code must retain the above copyright notice, this
          list of conditions and the following disclaimer.
        
        * Redistributions in binary form must reproduce the above copyright notice,
          this list of conditions and the following disclaimer in the documentation
          and/or other materials provided with the distrib

In [3]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer
import os
import re
import csv
import pandas as pd

C:\Users\jeanm\AppData\Roaming\Python\Python39\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import os
import re
import csv

In [5]:
import pandas as pd

In [6]:
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer

In [14]:
from filters import txt_to_string, filter_similar_entities

In [8]:
!pip show fuzzywuzzy

Name: fuzzywuzzy
Version: 0.18.0
Summary: Fuzzy string matching in python
Home-page: https://github.com/seatgeek/fuzzywuzzy
Author: Adam Cohen
Author-email: adam@seatgeek.com
License: GPLv2
Location: c:\program files\arcgis\pro\bin\python\envs\arcgispro-py3\lib\site-packages
Requires: 
Required-by: 


In [9]:
!pip install fuzzywuzzy

In [17]:
def entities_extractions(dossier):
    """
        Extrait les mots-clés de fichiers texte dans un dossier et enregistre les résultats dans des fichiers séparés.

        Cette fonction parcourt un dossier donné, extrait le contenu de chaque fichier texte, puis utilise
        le modèle KeyBERT pour extraire les mots-clés les plus pertinents. Les résultats sont ensuite sauvegardés
        dans un fichier texte correspondant à chaque fichier analysé.

        Paramètres:
        -----------
        dossier : str
            Chemin vers le dossier contenant les fichiers texte (.txt).

        Retour:
        -------
        None : La fonction ne retourne rien, mais crée des fichiers texte contenant les mots-clés et leurs scores.
    """
    odd_number = re.search(r'\d+', dossier).group()
    print("------------------------------------------------------------------------------------------------")
    print(f"Début de traitement de l'ODD {odd_number}...\n")
    sentence_model = SentenceTransformer("all-MiniLM-L6-v2")

    # Liste des noms de fichiers texte dans le dossier
    # J'utilise la condition supplementaire pour éviter de selectionner les fichiers traités
    txt_files = [fichier for fichier in os.listdir(dossier)
                 if fichier.endswith(".txt") and "keywords" not in fichier]

    # Liste des données extraites
    extracted_data = []

    for txt_file in txt_files:
        # Convertir le fichier texte en une chaîne de caractères
        texte = txt_to_string(f"{dossier}/{txt_file}")
        # Ajouter les données extraites dans la liste
        extracted_data.append((txt_file, texte))
        # print(extracted_data)
        # break

    # # Récupérer les numéros des cibles
    # target_numbers = []
    # for data in extracted_data:
    #     if data[0][12:14] not in target_numbers:
    #         target_numbers.append(data[0][12:14])
    # # print(target_numbers)

    keywords_ODD = []
    for data in extracted_data:
        # Récupérer le fichier texte extrait
        doc = data[1]
        # Configurer du modèle KeyBert pour la recherche de mots/phrases clé(e)s
        kw_model = KeyBERT(model=sentence_model)
        keywords = kw_model.extract_keywords(
            doc,
            keyphrase_ngram_range=(1, 3),   # Taille de mots/phrases clé(e)s
            stop_words='english',           # Eliminer les mots usuels en anglais
            # use_maxsum=True,
            nr_candidates=20,              # Nombre d'éléments à étudier
            top_n=10                        # Nombre d'éléments choisis
        )
        # Suppression des entités similaires d'un point de vue orthographique
        entities_filtered = filter_similar_entities(keywords, threshold=80)

        # Filtrer les résultats par score
        filtered_keywords = [(kw[0], kw[1], data[0][12:14]) for kw in keywords if kw[1] > 0.4]        # Ajouter les mots-clés extraits et leurs scores dans la liste des mots-clés pour l'ODD
        keywords_ODD.extend(filtered_keywords)
        # Chemin de sortie du fichier
        metadata = data[0].replace(".txt","")
        output_path = f"./{dossier}/{metadata}_keywords.txt"
        with open(output_path, "w", encoding="utf-8") as f:
            f.write("Mots-clés extraits et scores :\n")
            f.write("----------------------------------------------------\n")
            for keyword in filtered_keywords:
                f.write(f"{keyword[0]} : {keyword[1]:.4f}\n")
            f.write("----------------------------------------------------\n")

    # Stockage en dataframe
    df = pd.DataFrame(keywords_ODD, columns=["Mots-cles", "Scores", "Cibles"])

    # Enregistrer le DataFrame complet en CSV
    output_path = f"./{dossier}/ODD{odd_number}_keywords.csv"
    df.to_csv(output_path, index=False)

    print(df.head())

    # output_path = f"./Keywords_odd/ODD{odd_number}.csv"
    output_path = f"../Keywords_odd/ODD{odd_number}.csv"
    with open(output_path, mode="w", newline='', encoding="utf-8") as f:
        writer = csv.writer(f)
        # Écrire l'en-tête
        writer.writerow(["Mots-cles", "Scores", "cibles"])
        # Écrire les mots-clés et leurs scores
        for keyword in keywords_ODD:
            writer.writerow([keyword[0], keyword[1],keyword[2]])


    print(f"...Fin de traitement de l'ODD {odd_number}")

    return df

In [18]:
print(entities_extractions("../MetaD/ODD01"))

------------------------------------------------------------------------------------------------
Début de traitement de l'ODD 01...

                     Mots-cles  Scores Cibles
0   international poverty line  0.7255     01
1      global absolute poverty  0.7141     01
2  international poverty lines  0.7136     01
3       global poverty poverty  0.7071     01
4        international poverty  0.7054     01
...Fin de traitement de l'ODD 01
                       Mots-cles  Scores Cibles
0     international poverty line  0.7255     01
1        global absolute poverty  0.7141     01
2    international poverty lines  0.7136     01
3         global poverty poverty  0.7071     01
4          international poverty  0.7054     01
..                           ...     ...    ...
155        monetary poor follows  0.6409     0b
156                monetary poor  0.6316     0b
157         monetary poor health  0.6047     0b
158      transfers poor monetary  0.5886     0b
159          cash transfers po